<a id=“title_ID”></a>
# JWST Pipeline Validation Testing Notebook: Spec2, srctype, MIRI
Instruments Affected: MIRI

## Introduction and summary of test being run

This notebook processes an image through calwebb_spec2 and examines the output of the flat_field step. The steps are as follow:

1) Set up data path and directory and image file names.

2) Set the appropriate header keywords to test.

2) Run data through the source_type step in calwebb_spec2.

3) Compare input to output SRCTYPE keyword

## Documentation
The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

##Data used

The data used in this test is a simulated MIRI image created using MIRISim. The documentation for MIRISim can be found here: https://wiki.miricle.org/bin/view/Public/MIRISim_Public?

Author: Mike Engesser

## Create Temporary Directory

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

<TemporaryDirectory '/var/folders/v1/hp5rmzsx5kzgr8md63p7ptfh0002br/T/tmpd7c4tmuf'>


## Set Up Import Statements

In [2]:
from astropy.io import fits, ascii
import pytest
import numpy as np
import numpy.ma as ma

import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.datamodels import RampModel, ImageModel, dqflags
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_spec2

from ci_watson.artifactory_helpers import get_bigdata
import crds
import os
import sys

2021-12-03 09:30:59,674 - stpipe - WARNING - /Users/mengesser/miniconda3/envs/jwst_validation_notebooks/lib/python3.8/site-packages/photutils/detection/findstars.py:33: AstropyDeprecationWarning: _StarFinderKernel was moved to the photutils.detection._utils module. Please update your import statement.
  warnings.warn(f'{name} was moved to the {deprecated[name]} module. '



In [3]:
# Specify CRDS locations and pmap
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

## Print pipeline version number 

In [4]:
jwst.__version__ 

'1.1.0'

## Read in data from Artifactory

In [5]:
file = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits')

## Define functions for validation and running pipeline

In [6]:
print('[*]', file=open('stpipe-log.cfg', "w"))
print('handler = file:pipeline.log', file=open('stpipe-log.cfg', "a"))
print('level = INFO', file=open('stpipe-log.cfg', "a"))

In [7]:
def run_spec2(im):
    """Creates and runs an instance of the JWST spec2 pipeline. """
    
    #only assign_wcs and srctype are run
    spec2 = calwebb_spec2.Spec2Pipeline()
    spec2.bkg_subtract.skip = True
    spec2.assign_wcs.skip = False
    spec2.imprint_subtract.skip = True
    spec2.msa_flagging.skip = True
    spec2.extract_2d.skip = True
    spec2.wavecorr.skip = True
    spec2.srctype.skip = False
    spec2.straylight.skip = True
    spec2.fringe.skip = True
    spec2.pathloss.skip = True
    spec2.barshadow.skip = True
    spec2.photom.skip = True
    spec2.resample_spec.skip = True
    spec2.cube_build.skip = True
    spec2.extract_1d.skip = True
    spec2.flat_field.skip = True
    spec2.save_results = True
        
    spec2.run(im)

In [8]:
def test_src_type(input_file, im1):
    """Passes data to spec2 pipeline and records the source type keyword of the output"""
    
    #run through spec2
    run_spec2(im1)
    
    #get the output file name
    cal_file = input_file.replace('rate.fits', 'cal.fits')
    
    #create an image model of the output and return the source type keyword
    im2 = ImageModel(cal_file)
    
    # Get the header source type value as well
    hdu=fits.open(cal_file)
    
    return im2.meta.target.source_type,hdu[1].header['SRCTYPE']

## Create an image model and test source type keywords

This test passes if none of the assertions return an error

In [9]:
im = ImageModel(file)

In [10]:
# Test the default case where no external information which should return EXTENDED
im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2021-12-03 09:31:05,760 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-12-03 09:31:05,762 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-12-03 09:31:05,765 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-12-03 09:31:05,767 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-12-03 09:31:05,768 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-12-03 09:31:05,770 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-12-03 09:31:05,774 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-12-03 09:31:05,776 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-12-03 09:31:05,777 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-12-03 09:31:05,779 - stpipe.Spec2Pipeline.master_

In [11]:
# Test the case where APT specified 'POINT'
im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

im.meta.target.source_type_apt = 'POINT'
    
src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2021-12-03 09:31:14,398 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-12-03 09:31:14,399 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-12-03 09:31:14,401 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-12-03 09:31:14,403 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-12-03 09:31:14,405 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-12-03 09:31:14,407 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-12-03 09:31:14,411 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-12-03 09:31:14,412 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-12-03 09:31:14,414 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-12-03 09:31:14,415 - stpipe.Spec2Pipeline.master_

In [12]:
# Test the case where APT specified 'EXTENDED'
im.meta.target.source_type_apt = 'EXTENDED'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2021-12-03 09:31:21,579 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-12-03 09:31:21,581 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-12-03 09:31:21,584 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-12-03 09:31:21,585 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-12-03 09:31:21,587 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-12-03 09:31:21,589 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-12-03 09:31:21,593 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-12-03 09:31:21,595 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-12-03 09:31:21,597 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-12-03 09:31:21,599 - stpipe.Spec2Pipeline.master_

In [13]:
# If the background target keyword is set, it should always returns EXTENDED
# regardless of what APT said

im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = True

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2021-12-03 09:31:28,659 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-12-03 09:31:28,661 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-12-03 09:31:28,663 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-12-03 09:31:28,665 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-12-03 09:31:28,666 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-12-03 09:31:28,669 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-12-03 09:31:28,673 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-12-03 09:31:28,674 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-12-03 09:31:28,677 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-12-03 09:31:28,679 - stpipe.Spec2Pipeline.master_

In [14]:
# If no information from APT, but dither keywords suggest a point source it should
# return POINT

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'POINT-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2021-12-03 09:31:38,438 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-12-03 09:31:38,440 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-12-03 09:31:38,442 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-12-03 09:31:38,444 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-12-03 09:31:38,445 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-12-03 09:31:38,447 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-12-03 09:31:38,451 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-12-03 09:31:38,453 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-12-03 09:31:38,455 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-12-03 09:31:38,458 - stpipe.Spec2Pipeline.master_

In [15]:
# If no information from APT, but dither keywords suggest an extended source it should
# return EXTENDED

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'EXTENDED-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2021-12-03 09:31:49,262 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-12-03 09:31:49,264 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-12-03 09:31:49,267 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-12-03 09:31:49,268 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-12-03 09:31:49,269 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-12-03 09:31:49,271 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-12-03 09:31:49,277 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-12-03 09:31:49,279 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-12-03 09:31:49,281 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-12-03 09:31:49,283 - stpipe.Spec2Pipeline.master_

In [16]:
# If the observation is a TSO, return POINT

im.meta.observation.bkgdtarg = False
im.meta.visit.tsovisit = True
im.meta.exposure.nints = 3

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2021-12-03 09:31:56,616 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-12-03 09:31:56,618 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-12-03 09:31:56,620 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-12-03 09:31:56,622 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-12-03 09:31:56,624 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-12-03 09:31:56,626 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-12-03 09:31:56,630 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-12-03 09:31:56,632 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-12-03 09:31:56,635 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-12-03 09:31:56,637 - stpipe.Spec2Pipeline.master_

In [17]:
print('No problems found.')

No problems found.
